In [32]:
import numpy as np
from sklearn import preprocessing
import copy

In [33]:
def activation(x):
    return 1 / (1 + np.exp(-x))

def dactivation(x):
    return np.exp(-x)/((1+np.exp(-x))**2)

In [250]:
# MLP osztály létrehozása.
class MLP:
    
    # A hálózat inicializálása az argumentumként megadott méretek alapján.
    def __init__(self, *args):
        # random seed megadása
        np.random.seed(123)
        # A hálózat formája (rétegek száma), amely megegyezik a paraméterek számával
        self.shape = args
        n = len(args)
        # Rétegek létrehozása
        self.layers = []
        # Bemeneti réteg létrehozása (+1 egység a BIAS-nak)
        self.layers.append(np.ones(self.shape[0]+1))
        # Rejtett réteg(ek) és a kimeneti réteg létrehozása
        for i in range(1,n):
            self.layers.append(np.ones(self.shape[i]))
        # Súlymátrix létrehozása
        self.weights = []
        for i in range(n-1):
            self.weights.append(np.zeros((self.layers[i].size,
                                         self.layers[i+1].size)))
        # dw fogja tartalmazni a súlyok utolsó módosításait (később pl. a momentum módszer számára)
        self.dw = [0,]*len(self.weights)
        # Súlyok újrainicializálása
        self.reset()
    
    # Súlyok újrainicializálási függvényének definiálása
    def reset(self):
        for i in range(len(self.weights)):
            # véletlen számok [0,1) tartományban 
            Z = np.random.random((self.layers[i].size,self.layers[i+1].size))
            # átskálázzuk a súlyokat -1..1 tartományba
            self.weights[i][...] = (2*Z-1)*1

    # A bemenő adatok végigküldése a hálózaton, kimeneti rétegig (forward propagation)
    def propagate_forward(self, data):
        # Bemeneti réteg beállítása (tanító adatok)
        self.layers[0][0:-1] = data
        # Az adatok végigküldése a bemeneti rétegtől az utolsó előtti rétegig (az utolsó ugyanis a kimeneti réteg).
        # A szigmoid aktivációs függvény használatával, mátrixszorzások alkalmazásával.
        # Az előadáson a "layers" változót jelöltük "a"-val.
        for i in range(1,len(self.shape)):
            self.layers[i][...] = activation(np.dot(self.layers[i-1],self.weights[i-1]))
        # Visszatérés a hálózat által becsült eredménnyel
        return self.layers[-1]

   
    
    def propagate_backward(self, target, lrate=0.1, opt='', reg='l2'):                 
        
        if opt=='momentum':            
            #HF2 start momentum
            mu=0.01
            deltas = []        
            error = -(target-self.layers[-1])  #default cost function                      
            delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
            deltas.append(delta)
            for i in range(len(self.shape)-2,0,-1):
                delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
                deltas.insert(0,delta)            

            for i in range(1,len(self.weights)):
                layer = np.atleast_2d(self.layers[i])
                delta = np.atleast_2d(deltas[i])            
                
                
                dw = -mu*np.dot(layer.T,delta)+lrate*self.dw[i-1] #momentum function: dw(t)=-mu*(dC/dw)+lrate*dw(t-1)
                # updating weights
                self.weights[i] += dw 

                # new weights
                self.dw[i] = dw                           
             #HF2 end momentum  
        elif reg != None:
            mu=0.01
            #HF2 start l1reg
            if reg=='l1':
                lambda1=0.01
                C0=0
                deltas = []
                
                summ=0
                for dim in range(len(self.weights)):
                    summ+=np.sum(np.abs(self.weights[dim]))
                error = C0+lambda1*summ #C=C0+lambda1*summa(|w|)
                delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
                deltas.append(delta)
                for i in range(len(self.shape)-2,0,-1):
                    delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
                    deltas.insert(0,delta)            
                for i in range(1,len(self.weights)-1):
                    layer = np.atleast_2d(self.layers[i])
                    delta = np.atleast_2d(deltas[i])
                                                                         
                    # new weights
                    self.weights[i+1] = self.weights[i]-mu*np.dot(layer.T,delta)-mu*lambda1*np.sign(self.weights[i]) # w(t+1)=w(t)-mu*(dC/dW)-mu*lambda1*signum(w(t))                                 
            #HF2 end l1regy
            
            #HF2 start l2reg
            if reg=='l2':
                lambda2=0.01
                C0=0
                deltas = []
                summ=0
                for dim in range(len(self.weights)):
                    summ+=np.sum(np.square(self.weights[dim]))
                error = C0+lambda2*summ*0.5 #C=C0+lambda2*summa(w^2)*0.5
                delta = np.multiply(error,dactivation(np.dot(self.layers[-2],self.weights[-1])))
                deltas.append(delta)
                for i in range(len(self.shape)-2,0,-1):
                    delta=np.dot(deltas[0],self.weights[i].T)*dactivation(np.dot(self.layers[i-1],self.weights[i-1]))
                    deltas.insert(0,delta)            
                for i in range(1,len(self.weights)-1):
                    layer = np.atleast_2d(self.layers[i])
                    delta = np.atleast_2d(deltas[i])
                                                                         
                    # new weights
                    self.weights[i+1] = self.weights[i]-mu*np.dot(layer.T,delta)-mu*lambda2*self.weights[i] # w(t+1)=w(t)-mu*(dC/dW)-mu*lambda2*w(t)                                  
            #HF2 end l1reg
        
        
        

        return (error**2).sum()

In [251]:
def learn(network, X, Y, valid_split, test_split, epochs=20, lrate=0.1):

        # train-validation-test minták különválasztása
        X_train = X[0:int(nb_samples*(1-valid_split-test_split))]
        Y_train = Y[0:int(nb_samples*(1-valid_split-test_split))]
        X_valid = X[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
        Y_valid = Y[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
        X_test  = X[int(nb_samples*(1-test_split)):]
        Y_test  = Y[int(nb_samples*(1-test_split)):]
    
        # standardizálás
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_valid = scaler.transform(X_valid)
        X_test  = scaler.transform(X_test)
    
        # ugyanolyan sorrendben keverjük be a bemeneteket és kimeneteket, a három külön adatbázisra
        randperm = np.random.permutation(len(X_train))
        X_train, Y_train = X_train[randperm], Y_train[randperm]
        randperm = np.random.permutation(len(X_valid))
        X_valid, Y_valid = X_valid[randperm], Y_valid[randperm]
        randperm = np.random.permutation(len(X_test))
        X_test, Y_test = X_test[randperm], Y_test[randperm]
        
        best_valid_err = np.inf
        es_counter = 0 # early stopping counter
        best_model = network
    
        # Tanítási fázis, epoch-szor megyünk át 1-1 véltelenszerűen kiválasztott mintán.
        for i in range(epochs):
            # Jelen megoldás azt a módszert használja, hogy a megadott 
            # tanító adatokon végigmegyünk és minden elemet először végigküldünk
            # a hálózaton, majd terjeszti vissza a kapott eltérést az
            # elvárt eredménytől. Ezt hívjuk SGD-ek (stochastic gradient descent).
            train_err = 0
            for k in range(X_train.shape[0]):
                network.propagate_forward( X_train[k] )
                train_err += network.propagate_backward( Y_train[k], lrate )
            train_err /= X_train.shape[0]

            # validációs fázis
            valid_err = 0
            o_valid = np.zeros(X_valid.shape[0])
            for k in range(X_valid.shape[0]):
                o_valid[k] = network.propagate_forward(X_valid[k])
                valid_err += (o_valid[k]-Y_valid[k])**2
            valid_err /= X_valid.shape[0]

            print("%d epoch, train_err: %.4f, valid_err: %.4f" % (i, train_err, valid_err))

        # Tesztelési fázis
        print("\n--- TESZTELÉS ---\n")
        test_err = 0
        o_test = np.zeros(X_test.shape[0])
        for k in range(X_test.shape[0]):
            o_test[k] = network.propagate_forward(X_test[k])
            test_err += (o_test[k]-Y_test[k])**2
            print(k, X_test[k], '%.2f' % o_test[k], ' (elvart eredmeny: %.2f)' % Y_test[k])
        test_err /= X_test.shape[0]

    

In [252]:
network = MLP(2,10,1)

In [253]:
nb_samples=1000
X = np.zeros((nb_samples,2))
Y = np.zeros(nb_samples)
for i in range(0,nb_samples,4):
    noise = np.random.normal(0,1,8)
    X[i], Y[i] = (-2+noise[0],-2+noise[1]), 0
    X[i+1], Y[i+1] = (2+noise[2],-2+noise[3]), 1
    X[i+2], Y[i+2] = (-2+noise[4],2+noise[5]), 1
    X[i+3], Y[i+3] = (2+noise[6],2+noise[7]), 0

In [254]:
network.reset()
learn(network, X, Y, 0.2, 0.1)

0 epoch, train_err: 0.0050, valid_err: 0.2821
1 epoch, train_err: 0.0050, valid_err: 0.2821
2 epoch, train_err: 0.0050, valid_err: 0.2821
3 epoch, train_err: 0.0050, valid_err: 0.2821
4 epoch, train_err: 0.0050, valid_err: 0.2821
5 epoch, train_err: 0.0050, valid_err: 0.2821
6 epoch, train_err: 0.0050, valid_err: 0.2821
7 epoch, train_err: 0.0050, valid_err: 0.2821
8 epoch, train_err: 0.0050, valid_err: 0.2821
9 epoch, train_err: 0.0050, valid_err: 0.2821
10 epoch, train_err: 0.0050, valid_err: 0.2821
11 epoch, train_err: 0.0050, valid_err: 0.2821
12 epoch, train_err: 0.0050, valid_err: 0.2821
13 epoch, train_err: 0.0050, valid_err: 0.2821
14 epoch, train_err: 0.0050, valid_err: 0.2821
15 epoch, train_err: 0.0050, valid_err: 0.2821
16 epoch, train_err: 0.0050, valid_err: 0.2821
17 epoch, train_err: 0.0050, valid_err: 0.2821
18 epoch, train_err: 0.0050, valid_err: 0.2821
19 epoch, train_err: 0.0050, valid_err: 0.2821

--- TESZTELÉS ---

0 [-0.6956089  -1.37664792] 0.23  (elvart eredmeny